In [3]:
# import webget
import pandas as pd
import numpy as np
import tensorflow as tf

# webget.download('https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data')
# webget.download('https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data')

train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data'
test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data'

col_names = {0:'Suit of card #1', 1:'Rank of card #1',
             2:'Suit of card #2', 3:'Rank of card #2',
             4:'Suit of card #3', 5:'Rank of card #3',
             6:'Suit of card #4', 7:'Rank of card #4',
             8:'Suit of card #5', 9:'Rank of card #5',
             10:'Poker Hand'}

# train = pd.read_csv('poker-hand-training-true.data', header=None)
train = pd.read_csv(train_url, header=None)
train.rename(col_names, axis='columns', inplace=True)

# test = pd.read_csv('poker-hand-testing.data', header=None)
test = pd.read_csv(test_url, header=None)
test.rename(col_names, axis='columns', inplace=True)

x_train = train.iloc[:,0:10]
y_train = train.iloc[:,10]
x_test = train.iloc[:,0:10]
y_test = train.iloc[:,10]

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(10,)))
model.add(tf.keras.layers.Dense(2048, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train.values, y_train.values, epochs=1000)
model.evaluate(x_test.values, y_test.values)

In [ ]:
model.save('my_model.h5')

In [4]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()
loss, acc = new_model.evaluate(x_test.values, y_test.values)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 2048)              22528     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                20490     
Total params: 43,018
Trainable params: 43,018
Non-trainable params: 0
_________________________________________________________________
25010/25010 [==============================] - 3s 114us/sample - loss: 0.0272 - acc: 0.9964
Restored model, accuracy: 99.64%


In [18]:
predictions = new_model.predict_proba(x_test)
vals = ['Nothing', 'One Pair', 'Two Pairs', 'Three of a kind', 'Straight',
        'Flush', 'Full house', 'Four of a kind', 'Straight flush', 'Royal flush']
count = 0
fails = []
for x in range(len(predictions)):
    pred = np.argmax(predictions[x])
    real = y_test.loc[x]
    if pred == real:
        count += 1
    else:
        fails.append((pred,real))

for x in sorted(list(set(fails)), key=lambda x: x[0]):
    print('prediction: ' + vals[x[0]], 'real: ' + vals[x[1]])
print('acc', count/len(predictions))

prediction: Nothing real: One Pair
prediction: Nothing real: Straight
prediction: One Pair real: Two Pairs
prediction: One Pair real: Straight
prediction: Two Pairs real: Full house
prediction: Two Pairs real: One Pair
prediction: Two Pairs real: Three of a kind
prediction: Three of a kind real: Two Pairs
prediction: Three of a kind real: Full house
prediction: Straight real: One Pair
prediction: Straight real: Nothing
acc 0.9964414234306278
